In [45]:
!python csimodel.py              # shape 500 biderectional acc = 96

Error! Correct Command: python3 csimodel.py Dataset_folder_path
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 90)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 500, 400)         465600    
 l)                                                              
                                                                 
 atten_layer (AttenLayer)    (None, 400)               160800    
                                                                 
 dense (Dense)               (None, 7)                 2807      
                                                                 
Total params: 629,207
Trainable params: 629,207
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30

50/50 [==============================] - 29s 

2023-04-10 23:11:48.835345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 23:11:49.390152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2476 MB memory:  -> device: 0, name: Quadro P1000, pci bus id: 0000:65:00.0, compute capability: 6.1
2023-04-10 23:11:51.644448: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1146600000 exceeds 10% of free system memory.
2023-04-10 23:11:56.165039: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


In [52]:
!python csimodel.py     # shape 500 conv1d  acc = 97

Error! Correct Command: python3 csimodel.py Dataset_folder_path
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 90, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 498, 88, 3)        30        
                                                                 
 flatten (Flatten)           (None, 131472)            0         
                                                                 
 dense (Dense)               (None, 7)                 920311    
                                                                 
Total params: 920,341
Trainable params: 920,341
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30

50/50 [==============================] - 6s 77ms/step - loss: 40.2509 - accuracy: 0.2915 - val_loss: 3.4528 - v

2023-04-11 09:59:57.353212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 09:59:57.882279: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2780 MB memory:  -> device: 0, name: Quadro P1000, pci bus id: 0000:65:00.0, compute capability: 6.1
2023-04-11 09:59:58.881434: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1146600000 exceeds 10% of free system memory.
2023-04-11 10:00:00.175530: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


In [2]:
from npz_loader import *
cfg = CSIModelConfig()
numpy_tuple = cfg.load_csi_data_from_files(('X_bed.npz', 'X_fall.npz', 'X_pickup.npz', 'X_run.npz', 'X_sitdown.npz', 'X_standup.npz', 'X_walk.npz'))
x_bed, y_bed, x_fall, y_fall, x_pickup, y_pickup, x_run, y_run, x_sitdown, y_sitdown, x_standup, y_standup, x_walk, y_walk = numpy_tuple
x_train, y_train, x_valid, y_valid = train_valid_split(
    (x_bed, x_fall, x_pickup, x_run, x_sitdown, x_standup, x_walk),
    train_portion=0.9, seed=379)

In [11]:
model = modeler(win_len=125)           # Central Model win len 125 acc 99.7 and val acc 89
model.fit(
    x_train,
    y_train,
    batch_size=128, epochs=100,
    validation_data=(x_valid, y_valid),
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('best_conv.hdf5',
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            save_weights_only=False)
        ])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 125, 90, 1)]      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 123, 88, 3)        30        
                                                                 
 flatten_2 (Flatten)         (None, 32472)             0         
                                                                 
 dense_2 (Dense)             (None, 7)                 227311    
                                                                 
Total params: 227,341
Trainable params: 227,341
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
52/52 [==============================] - 2s 23ms/step - loss: 6.7196 - accuracy: 0.2436 - val_loss: 1.9107 - val_accuracy: 0.3706
Epoch 2/100
52/52 [========================

In [22]:
n_nodes = 10            # Created Nodes for federated learning _ Validation
n_ration = len(x_train)/n_nodes
nodes_dict_train = {}
for i in range(1,n_nodes+1):
    n_ration_f = int(i*n_ration)
    n_ration_b = int((i-1)*n_ration)
    x_train_ = x_train[n_ration_b:n_ration_f,:,:]
    y_train_ = y_train[n_ration_b:n_ration_f,:]
    nodes_dict_train[i] = x_train_,y_train_
    print(n_ration_b,n_ration_f)

0 665
665 1330
1330 1995
1995 2660
2660 3325
3325 3990
3990 4655
4655 5320
5320 5985
5985 6650


In [23]:
n_nodes = 10        # Created Nodes for federated learning _ Validation
n_ration = len(x_valid)/n_nodes
nodes_dict_valid = {}
for i in range(1,n_nodes+1):
    n_ration_f = int(i*n_ration)
    n_ration_b = int((i-1)*n_ration)
    x_valid_ = x_valid[n_ration_b:n_ration_f,:,:]
    y_valid_ = y_valid[n_ration_b:n_ration_f,:]
    nodes_dict_valid[i] = x_valid_,y_valid_
    print(n_ration_b,n_ration_f)

0 74
74 148
148 222
222 296
296 371
371 445
445 519
519 593
593 667
667 742


In [24]:
# Weight scaling for global averaging
def scale_model_weights(weight, scalar):
    scalar = 1/scalar
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

# weight addition for appending to global model
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K



# Parameter Setting
nEpochs = 10  # number of local rounds
comms_round =  10  # number of global round 
k=5  # Number of mandals choosen
earlystop = EarlyStopping(patience=10)

# Global Model Initialization
global_model = modeler(win_len=125)
opt=[]

#randomize client data - using keys

client_names= list(nodes_dict_train.keys())

for comm_round in range(comms_round):
            
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list() 
    #loop through each client and create new local model
    for client in opt:
        local_model = modeler(win_len=125)
        local_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss='categorical_crossentropy', 
            metrics=['accuracy'])
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # Data Seperation for min and max input
        (x_train,y_train),(x_valid,y_valid) = nodes_dict_train[client],nodes_dict_valid[client]
        
        history_local = local_model.fit(
                                        x_train,y_train,
                                        batch_size=128, epochs=100,
                                        validation_data=(x_valid, y_valid),
                                        callbacks=[
                                            tf.keras.callbacks.ModelCheckpoint('best_conv.hdf5',monitor='val_accuracy',
                                                                                save_best_only=True,save_weights_only=False)])
        print('Epoch done for node {} '.format(client))

        #scale the model weights and add to list
        scaled_weights = scale_model_weights(local_model.get_weights(), scalar=k)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    print("\n")
    print("Round {} Done for all the nodes".format(comm_round))  

    #update global model 
    global_model.set_weights(average_weights)

    # test global model and print out metrics after each communications round